In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from fuzzywuzzy import fuzz
from sklearn.cluster import AffinityPropagation
import os
import groupby_toolz
import spacy #https://spacy.io/usage/linguistic-features
# install spacy here https://spacy.io/usage
# import en_core_web_sm
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from IPython.display import display
pd.options.display.max_columns = None
from decimal import *
TWOPLACES = Decimal(10) ** -2
# https://stackoverflow.com/questions/21903842/how-do-i-compare-two-strings-in-python

#Colors
import webcolors
import math

In [3]:
#######################################################################  Syndication Data  ##########################################################################
def col_group(df, fuzzy = 'partial',return_dict = False):
    '''
    Affinity Clustering to sort column names into clusters based on various fuzzy match
    param:
    =======
    df: Pandas DataFrame, syndicated DataFrame
    fuzzy: str, default value is partial, to select fuzzy ratio type: ['ratio','partial','token_sort','token_set','partial_sort','partial_set'] 
    return_dict: bool, if False, prints output. If True, returns a dict with each cluster. Default value False 
    '''
    # Modified from: https://bit.ly/3vRuE1y
    assert isinstance(return_dict, bool), "return_dict argument must be boolean"
    assert fuzzy in ['ratio','partial','token_sort','token_set','partial_sort','partial_set'],\
    "fuzzy must be in 'ratio','partial','token_sort','token_set','partial_sort', or 'partial_set'"
    words = np.asarray(df.columns)
    # Pick a fuzzy ratio
    if fuzzy == 'ratio':
        lev_similarity = np.array([[fuzz.ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial':
        lev_similarity = np.array([[fuzz.partial_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'token_sort':
        lev_similarity = np.array([[fuzz.token_sort_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'token_set':
        lev_similarity = np.array([[fuzz.token_set_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial_sort':
        lev_similarity = np.array([[fuzz.partial_token_sort_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial_set':
        lev_similarity = np.array([[fuzz.partial_token_set_ratio(w1,w2) for w1 in words] for w2 in words])
    # Affinity clustering
    affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
    affprop.fit(lev_similarity)
    # Simply print output
    if return_dict == False:
        for cluster_id in np.unique(affprop.labels_):
            exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
            cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
            cluster_str = ", ".join(cluster)
            print(" - *%s:* %s" % (exemplar, cluster_str))
            print('\b')
    # Returns a dictionary instead
    elif return_dict == True:
        dict_ = defaultdict()
        for cluster_id in np.unique(affprop.labels_):
            exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
            cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
            cluster_str = ", ".join(cluster)
            dict_[exemplar]=[f'{(exemplar, cluster_str)}']
        return dict_
# Print out values from two columns that have uncommon values
def print_att(df,one,two):
    for i in range(len(df)):
        Str1 = str(df[f'{one}'][i])
        Str2 = str(df[f'{two}'][i])
        Partial_Ratio = fuzz.token_set_ratio(Str1,Str2) 
        if Partial_Ratio<100:
            print(f"{str(df[f'{one}'][i])}, {str(df[f'{two}'][i])}")
            
            
def mult_func_two(df,one,two):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])
def mult_func_three(df,one,two,three):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}',    f'{three}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])
def mult_func_four(df,one,two,three,four):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}',    f'{three}',f'{four}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])



#######################################################################    Color    ##########################################################################
colorSplitRegex = r'-| |/|\\|,'
# G E T   A   C O L O R   N A M E
# get the closest color from an rgb tuple
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]
# get the color name either from webcolors or closest_color
def get_colour_name(requested_colour):
    try:
        closest_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
    return closest_name.title()
# G E T   A   C O L O R   V A L U E
# convert a string to an rgb tuple
def parseColor(string):
    try:
        # convert the color string to rgb, default spec is css3
        return webcolors.name_to_rgb(string)
    except Exception:
        pass
# parse raw data for strings we can match on
# if the string has a dash, slash, or space try each color individually
# and average out the RGB values with a simple a+b+c/n forumla
def parseColorData(data):
    # init an empty array for color values
    colorValues = []
    # split it on dashes, spaces, slashes, or commas
    if re.split(colorSplitRegex, data):
        for color in re.split(colorSplitRegex, data):
            colorValues.append(parseColor(color))
    # if it can't be split just full send it
    else:
        colorValues.append(parseColor(color))
    # remove None values
    colorValues = [
        colorValue for colorValue in colorValues if colorValue is not None]
    if not len(colorValues):
        return data
    # find the average color and convert it to a color name
    #######################################################
    # init an average color value
    averageColor = [0, 0, 0]
    for colorValue in colorValues:
        for index, value in enumerate(colorValue):
            averageColor[index] += value
    averageColor = [math.floor(value / (len(colorValues) or 1))
                    for value in averageColor]
    # just change this to a return to run this function
    return get_colour_name(averageColor)





#######################################################################    Clean/Merge Data    ##########################################################################

# Finds the first digit (basically skips over words and letters to find the first number)
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

# i finds the first digit: if it equals zero then there is not a digit, if it equals the end then leave it
def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

            
# NATURAL SORT
######################################################################
# sort the number naturally so 5 will come before 15
def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))
# x['matches']= x['matches'].apply(lambda x: sorted(x, key=natural_key))



#------------atof and natural_keys are complementary-------------
# atof and natural_keys: Will naturally sort 5 infront of 15
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]
#########################################################################





# REPLACE
#########################################################################
# # Replace specific text in a string: Exmapel1  to  Example1
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# replacement={'Exmapel1':'Example1',    }
# df= df.apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)


def replace_all(text, dic):
    '''
    Given a text and replacement dictionary, performs multiple str.replace()
    Arguments:
    ==========
    text: str, text to perform str.replace()
    dic: dict, where each value is the string to replace, and each key is the string to replace with
    '''
    for key, value in dic.items():
        for v in value:
            text = text.replace(v, key)
    return text
# power['min_temp_sj']=power['min_temp_sj'].apply(lambda x: replace_all(x,{'':['Ã‚Â°F']}))


# Replace all letters and some special characters with a blank
# data['max']=data['max'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(measure)))
#########################################################################




# FRACTIONS
#########################################################################
# Convert a fraction that does not have an int in front of the fraction "1/3"
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))
#########################################################################





# CONVERTING FRACTIONS, WHOLE NUMBERS, AND NUMBERS WITH LETTERS
#########################################################################
#Converts the number to a decimal if it has a decimal number but leaves off .0 if it is an int. Be sure to conver it back to a string and add -"in"
def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()
# from decimal import *
# TWOPLACES = Decimal(10) ** -2
# tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))



 
# converts the value to a float if possible and skips the value if not   
def convert_float(val):
    try:
        return float(val)
    except:
        return str(val)
    
    
    

# Definition to round a number
def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x

    
    
# Excel might convert 8-1/2 to 08/01/2002 and we need to conver this back into 8.5  
def date_to_decimal(numbers):
    if numbers.find('/') > -1:
        try:
    #         print(numbers)
    #         numbers = literal_eval(numbers)
    #         print(numbers)
            numbers=str(numbers)
            one, two,three = numbers.split('/')
            tmp = float(two)/float(three[-1])
            num = int(one) + tmp
            return str(num)
        except:
            return numbers
    else:
#         print(numbers)
        return numbers

# Convert a fraction "1/4" that is written as a date "1-4-2021"
def remove_basic_date(numbers):
    if numbers.find('-2021') > -1:
        first, second, year = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
# Convert a fraction "1/4" that is written as a date "2021-01-04"
def remove_basic_date_year_first(numbers):
    if numbers.find('2021-') > -1:
        year, first, second = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
# Convert a fraction "1/4" that is written as a date "1/4/2021"    
def remove_basic_date_year_first_slash(numbers):
    if numbers.find('/2021') > -1:
        year, first, second = numbers.split('/')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers



# Just merging everything into one function
def final_done(df,col):
    return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|,|\(|\)','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda x: remove_basic_date(x)).apply(lambda x: remove_basic_date_year_first(x)).apply(lambda x: remove_basic_date_year_first_slash(x)).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: round_string_float(x))
    
# final_done(power, 'height_sj_one')
#########################################################################









# TEMP RANGES
#########################################################################
# Converts Above 40 Degrees F  and Above 0
def above_freezing(numbers): 
    if numbers.find('above freezing') > -1:   
        numbers=0
        return numbers
    if numbers.find('above ') > -1:   
        numbers=re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(numbers))
        return numbers
    else:
        return numbers
# data['min']=data['min'].apply(lambda x: above_freezing(x))
    
#Splits temp ranges: 60 to 100 Degrees F, -30 to 200 Degrees F, 50-100 Degrees
def split_temp(data, col, dimension):
#     data['matches']=data['matches'].str.get(0) # Use this to for arrays
    data[col]=data[col].apply(lambda x:x.lower()) #sometimes x or to is capitalized like X or To and it wont split
    data[['min_'+dimension, 'max_'+dimension]]=data[col].str.split(' to |(?<=\d)\s?-|max',expand=True,) # have to split on "to" as well as ranges "50-100" but have to keep "-30" from "-30 to 200" 
    data['min_'+dimension]=data['min_'+dimension].apply(lambda x: above_freezing(x)).apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(measure)))
    data['max_'+dimension]=data['max_'+dimension].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°','',str(measure)))
# split_temp(power, 'operating_temp_sj')   

#########################################################################

In [4]:
f_path=r'G:\Shared drives\GroupBy Public\Customer Success\.Enrich\ScrapedData\Motion - Syndication Deliverable\Final Delivery\_previously_sent\delivered_unified'
csvs = sorted([file for file in os.listdir(f_path) if file[-len('csv'):] == 'csv'])

In [5]:
electrical = pd.read_csv(f'{f_path}/{csvs[13]}', dtype=str)
print(len(electrical.columns.tolist())) #print number of columns before dropping zeros
zero_col=pd.DataFrame(electrical.notnull().sum())
zero_col=zero_col.reset_index()
zero_col.columns = ['col', 'not_null']
electrical.drop(zero_col[zero_col['not_null'].astype(str)<'1']['col'], axis =1, inplace = True)
print(len(electrical.columns.tolist())) # print number of relavent columns
new_col=zero_col[zero_col['not_null'].astype(str)>'1'][3:].reset_index(drop=True) # first 11 columns are the url/external_id/etc.. and we will not drop these
first=new_col['col'][0]
second=new_col['col']

420
381


# Electrical

In [6]:
power=electrical

In [7]:
col_group(power, fuzzy='partial', return_dict = False)
# print_att(power, 'Strength','Threadlocker_Strength',)
# mult_func_two(power,'S:Overall_Length', 'S:Overall_Length_In')

 - *Product_Name:* (aa)_Web Product Description, ANSI_Sprocket_Number, Hole_Saw_Product_Grouping, ProductURL, Product_Compatibility, Product_Name

 - *Angle:* Angle, Bushing_Required, Max_Angular_Misalignment, Opening_Angle_Deg, Operating_Temp, Single_End_Double_End

 - *Application:* Application, Application_Tools, Motor_Application, Primary_Saw_Application, Suitable_for_High_Temperature_Application

 - *Capacity:* Basket_Hitch_Capacity, Capacity, Choker_Hitch_Capacity, Expansion_Capability, Vertical_Hitch_Capacity

 - *Center Distance:* Center Distance, Center_distance_inch

 - *Depth:* Depth, Max_Cutting_Depth, Socket_Depth, Socket_Shape, Tether_Capable, Tether_Ready

 - *Height:* Base_to_Shaft_Center_Height_wStd_Base_Kit, Box_Height, Gross_Weight, Height, Height_In, Link_Plates_Height, Overall Height, Weight

 - *Length:* Approximate_Inside_Length, Bore Length, Box_Length, Datum_Length, EffectiveLength_in, EffectiveLength_mm, Effective_Length_in, Effective_Length_mm, Eye_Len

# Merge

In [ ]:
err

In [17]:
# convert all columns to a string to manipulate them
# power[second.to_list()]=power[second.to_list()].astype(str)

In [ ]:
zero_col=pd.DataFrame(power.notnull().sum())
zero_col=zero_col.reset_index()
zero_col.columns = ['col', 'not_null']
len(zero_col[zero_col['not_null'].astype(str)<'1']['col'])
power.drop(zero_col[zero_col['not_null'].astype(str)<'1']['col'], axis =1, inplace = True)
print(len(power.columns.tolist())) 
new_col=zero_col[zero_col['not_null'].astype(str)>'1'][3:].reset_index(drop=True) # first 11 columns are the url/external_id/etc.. and we will not drop these
first=new_col['col'][0]
second=new_col['col']

In [ ]:
new=pd.DataFrame()
new['package_new']=power[[


    


]].apply(lambda x: ','.join(x.dropna()), axis=1)
print(len(new['package_new'].explode().value_counts()))
# print(len(new['package_new']))
new['package_new'].explode().value_counts()[0:500]

In [ ]:
# power['strength_sj']=power[[
#  'Strength','Threadlocker_Strength',]].apply(lambda x: ",".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# power['shear_strength_sj'].explode().value_counts()

### Merge

In [18]:
power.drop([  


], axis=1, inplace=True)# 

### Delete

In [19]:
power.drop([
    
    

    

    

           ], axis=1, inplace=True)# 

In [ ]:
# str2Match = 'length'
# strOptions = second
# Ratios = process.extract(str2Match,strOptions, limit=14)
# for i in range(len(Ratios)):
#     print(f"'{Ratios[i][0]}',")